In [391]:
from copy import deepcopy
import math

In [457]:
from copy import deepcopy
class Etat : 
    def __init__(self, visited=[], tovisit=[], startingcity = '', currentcity='', costs=[], cost=0, cities=[], pere = Etat()):
        self.visited = visited #les villes deja visitees on pourrait utiliser une liste
        self.tovisit = tovisit # les villes a visiter, une liste pareil
        self.startingcity = startingcity # ville de départ qui devre etre la ville d'arrivee
        self.currentcity = currentcity # la ville ou on en est, on pourrait aussi faire self.visited.tail peut être
        self.costs = costs # une liste de liste [[a,b,3], [a,c,5], etc...]
        self.cost = cost # le coût de ce chemin à l'étape currentcity servira de g(n)
        self.cities = cities
        self.pere = pere # etat precedent
        
    def addpath(self, path): #path de la forme ['Paris','Lyon',45] et l'ajoute à self
        nouv = deepcopy(self)
        if (path[0] in nouv.visited):
            nouv.currentcity = path[1]
        else:
            nouv.currentcity = path[0]
        nouv.visited.append(path[0])
        nouv.visited.append(path[1])
        nouv.visited = list(set(nouv.visited))
        nouv.cost+= path[2]
        if (path[0] in nouv.tovisit ) : nouv.tovisit.remove(path[0]) # ce qui est visité n'est plus à visiter
        if (path[1] in nouv.tovisit ) :nouv.tovisit.remove(path[1])
        return nouv 

    def getmintree(self):
        cop = self.costs.copy()
        vis = self.visited
        edg = []
        cos = self.cost # totalcost
        index = 0
        if len(vis)==0: # to get started
            for i in range(len(cop)):
                for j in range(len(cop)):
                    if (cop[i][2]<=cop[j][2]) and (cop[index][2]>cop[i][2]):
                        index = i    
            cos+=cop[index][2] # le cout total
            edg.append(cop[index])
            vis.extend((cop[index][0],cop[index][1])) # on met les villes visited dans vis
        if len(vis) != 0:
            for k in range(len(self.cities)-len(vis)):
                index = 0
                for i in range(len(cop)):
                    if not((cop[i][0] in vis) and (cop[i][1] in vis)) and not((cop[i][0] not in vis) and (cop[i][1] not in vis)):
                        for j in range(len(cop)):
                            if not((cop[j][0] in vis) and (cop[j][1] in vis)) and not((cop[j][0] not in vis) and (cop[j][1] not in vis)): # ici on a l'ensemble frontière
                                if (cop[i][2]<=cop[j][2]) and (cop[index][2]>cop[i][2]):
                                    index = i
                vis.extend((cop[index][0],cop[index][1]))
                vis = list(set(vis)) #pour supprimer les doublons
                cos+=cop[index][2]
                edg.append(cop[index])
                self.currentcity = edg[-1][1]
                self.visited = vis
                self.cost = cos
                self.tovisit = list(set(self.tovisit)- set(self.visited))
        return edg

    def heuristic(self): # sera notre heuristic via calcul du minspantree
        cop = self.costs.copy()
        vis = self.visited
        edg = []
        cos = self.cost # totalcost
        index = 0
        frontier = []
        if len(vis)==0: # to get started
            for i in range(len(cop)):
                for j in range(len(cop)):
                    if (cop[i][2]<=cop[j][2]) and (cop[index][2]>cop[i][2]):
                        index = i    
            cos+=cop[index][2] # le cout total
            edg.append(cop[index])
            vis.extend((cop[index][0],cop[index][1])) # on met les villes visited dans vis
        if len(vis) != 0:
            for k in range(len(self.cities)-len(vis)):
                index = 0
                for i in range(len(cop)):
                    if not((cop[i][0] in vis) and (cop[i][1] in vis)) and not((cop[i][0] not in vis) and (cop[i][1] not in vis)):
                        if (k == 0): #la frontier pour le premier coup
                            frontier.append(cop[i]) # les edges encore candidats
                        for j in range(len(cop)):
                            if not((cop[j][0] in vis) and (cop[j][1] in vis)) and not((cop[j][0] not in vis) and (cop[j][1] not in vis)): # ici on a l'ensemble frontière
                                if (cop[i][2]<=cop[j][2]) and (cop[index][2]>cop[i][2]):
                                    index = i
                vis.extend((cop[index][0],cop[index][1]))
                #vis = list(set(vis)) #pour supprimer les doublons
                cos+=cop[index][2]
                edg.append(cop[index])
                self.currentcity = edg[-1][1]
                self.visited = vis
                #self.cost = cos
                self.tovisit = list(set(self.tovisit)- set(self.visited))
                
                new_f = []
                for elem in frontier:
                    if elem not in new_f:
                        new_f.append(elem)
                frontier = new_f # il y a toutes les frontières ici mais on peut voir à les avoir pour une certaine étape ou faire une liste de liste
        return (cos - self.cost)


    def frontier(self): # va retourner les etats frontier qui peuvent etre ajouté à l'arbre mais pas forcément dans astar comme on doit partie de la dernière ville visitée
        front = []
        vis = self.visited.copy()
        cop = self.costs.copy()
        for i in range(len(cop)):
            if not((cop[i][0] in vis) and (cop[i][1] in vis)) and not((cop[i][0] not in vis) and (cop[i][1] not in vis)): #la frontier pour le premier coup
                front.append(cop[i]) # les edges encore candidats
        new_f = []
        for elem in front:
            if elem not in new_f:
                new_f.append(elem)
        front = new_f
        return front

    def openfront(self):
        op = []
        vis = self.getvisited()
        cop = self.costs.copy()
        for i in range(len(cop)):
            if ((cop[i][0] == self.getcurrentcity()) and (cop[i][1] not in vis)) or ((cop[i][1] == self.getcurrentcity()) and (cop[i][0] not in vis)): #on veut que les coups suivants partent de la dernière ville visitée
                op.append(cop[i]) # les edges encore candidats
        return op


    def addvisited(self, city): # serves to add a city to the visited ones
        try:
            assert(city in self.tovisit)
            self.visited.append(city)
        except:
            print("Cette ville n'est pas dans la liste des villes encore disponibles \n Essayer la fonction ShowUnvisitedCities")

    def getvisited(self):
        return self.visited

    def getcosts(self):
        return self.costs

    def getcost(self):
        return self.cost

    def getcurrentcity(self):
        return self.currentcity

    def getstartingcity(self):
        return self.startingcity

    def getcities(self):
        return self.cities
    
    def getpere(self):
        return self.pere

    def show_unvisited_cities(self):
        print(f"There are the cities left to visit {self.tovisit}")

    def prettyprint(self):
        print('Villes déjà visitées :' + str(self.visited))
        print('Villes à visiter :' + str(self.tovisit))
        print('Ville de départ :' + self.startingcity)
        print('Villeactuelle :' + self.currentcity)
        print('Données :' + str(self.costs))
        print('Coût à cette étape :' + str(self.cost))
        print('Toutes les villes :' + str(self.cities))
        print('Villes visitées à l`\'étape précédente :' + str(self.pere.visited))
        

In [459]:
data = [['a','b',21],['a','c',9],['a','d',21],['a','e',1]
,['b','c',4],['b','d',8],['b','e',20]
,['c','d',6],['c','e',7]
,['d','e',11]]

datacomp = [['a','b',21],['a','c',9],['a','d',21],['a','e',1]
,['b','a',21],['b','c',4],['b','d',8],['b','e',20]
,['c','a',9],['c','b',4],['c','d',6],['c','e',7]
,['d','a',21],['d','b',8],['d','c',6],['d','e',11]
,['e','a',1],['e','b',20],['e','c',7],['e','d',11]]

origine = Etat(['a'],['a','b','c','d','e'], 'a', 'a', data, 0, ['a','b','c','d','e'], Etat())

nouveau = Etat(['a','e'],['c','b','d'], 'a','e', data, 1,['a','b','c','d','e'], Etat())
listage = []
neu = deepcopy(origine) # les differentes options qu'on a pour l'étape suivante
hihi = neu.addpath(['a','e',1])
haha = hihi.addpath(['e','c',7])
fr = haha.frontier()
val1 = math.inf
a = 0
ind = 0
for i in fr :
    val0 = (haha.addpath(i).heuristic()+haha.addpath(i).getcost()) # f = h + g
    if (val0 < val1):
        ind = a
        val1 = val0
    a+=1

huhu = haha.addpath(fr[ind])
huhu.prettyprint()
print('-------------------------------')
fr2 = huhu.frontier()
val2 =math.inf
a2 = 0
ind2 = 0
for i in fr2 :
    val0 = (haha.addpath(i).heuristic()+haha.addpath(i).getcost()) # f = h + g
    if (val0 < val2):
        ind2 = a2
        val2 = val0
    a2+=1
hehe = huhu.addpath(fr2[ind2])
hehe.prettyprint()

Villes déjà visitées :['a', 'b', 'e', 'c']
Villes à visiter :['d']
Ville de départ :a
Villeactuelle :b
Données :[['a', 'b', 21], ['a', 'c', 9], ['a', 'd', 21], ['a', 'e', 1], ['b', 'c', 4], ['b', 'd', 8], ['b', 'e', 20], ['c', 'd', 6], ['c', 'e', 7], ['d', 'e', 11]]
Coût à cette étape :12
Toutes les villes :['a', 'b', 'c', 'd', 'e']
Villes visitées à l`'étape précédente :[]
-------------------------------
Villes déjà visitées :['a', 'e', 'd', 'c', 'b']
Villes à visiter :[]
Ville de départ :a
Villeactuelle :d
Données :[['a', 'b', 21], ['a', 'c', 9], ['a', 'd', 21], ['a', 'e', 1], ['b', 'c', 4], ['b', 'd', 8], ['b', 'e', 20], ['c', 'd', 6], ['c', 'e', 7], ['d', 'e', 11]]
Coût à cette étape :20
Toutes les villes :['a', 'b', 'c', 'd', 'e']
Villes visitées à l`'étape précédente :[]


In [472]:
trace = Etat(['a'],['a','b','c','d','e'], 'a', 'a', data, 0, ['a','b','c','d','e'], Etat())
#trace = trace.addpath(['a','e',1])
#trace = trace.addpath(['e','c',7])
print(trace.openfront())
#trace = trace.addpath(['c','d',6])
#trace = trace.addpath(['d','a',21])
r1 = trace.heuristic() + trace.getcost()
print(r1)
#trace.prettyprint()

[['a', 'b', 21], ['a', 'c', 9], ['a', 'd', 21], ['a', 'e', 1]]
18


In [468]:
etat = Etat()
def astar(etat):
    op = [etat] #la frontière, à voire comment représenter ça, ce sera une liste d'états, à ranger dans l'ordre croissant de f = g + h ou alors avoir une fonction qui nous donne le min
    closed = [] # c'est les états validés
    closed.append(op [0]) # initialisation
    for i in closed[-1].openfront():#toujours l'initianisation
        op.append(i)
    cop = deepcopy(etat)
    ###
    indice = 0
    first = op[0].heuristique()+op[0].getcost()
    for i in range(len(op)): # on va prendre dans indice l'indice de ce qui nous intéresse dans op
        if (op[i].heuristique()+op[i].getcost())< first:
            first = op[i].heuristique()+op[i].getcost()
            indice = i
    closed.append(op.pop(indice))
    op.append(closed[-1].openfront()) ## faut ajouter des etats et pas des ['a','b',5]
    ###
    while (len(etat.getevisited()) != len(etat.getcities())): # tant qu'on a pas visité toutes les villes
        c = closed[-1] #le dernier etat rentré dans les closed
        fr = c.openfront() # la frontière de ce truc
        v1 = math.inf
        for i in fr:
            etat2 = deepcopy(etat)
            etat2.addpath(i)
            v0 = etat2.heuristic() + etat2.getcost() # f = h + g
            if (v0<v1):
                v1 = v0
                etatf = deepcopy(etat2) # on récupère l'état le plus intéressant parmis ceux de open ou de la frontière
        # trouver un moyen de retirer le truc de open
        closed.append(etatf)


In [ ]:
def aStar(self):
    start = self
    stop = self.startingcity
    #
    open_lst = set([start]) # on va y mettre les etats a etudier
    closed_lst = set([]) # on y met les etats etudie
    




    while len(open_lst) > 0:
        n = None

        for v in open_lst:

